In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# from model import Create_transfert_learning_Model
from processing.generator import Dataset_Generator
from processing.processor import Image_Processing

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#   tf.config.experimental.set_memory_growth(gpu, True)

gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

2022-11-25 15:56:36.678590: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 15:56:36.714123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 15:56:36.745359: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 15:56:36.745701: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [2]:
print(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Memory Usage",tf.config.experimental.get_memory_usage('GPU:0')* 1e-9, "GB")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1
Instructions for updating:
Use tf.config.experimental.get_memory_info(device)['current'] instead.
Memory Usage 0.0 GB


2022-11-25 15:56:37.621612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 15:56:37.621870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 15:56:37.622003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 15:56:37.625238: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 15:56:37.625704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [3]:
2341994496 * 1e-9, "GB"

(2.3419944960000003, 'GB')

In [4]:
900157440 * 1e-9

0.9001574400000001

In [5]:
Generator = Dataset_Generator(nbr_augmentation=1)

In [6]:
X_train, Y_train,\
X_dev, Y_dev,\
X_test, Y_test = Generator(healthy_dir_name='leaves_img/sain',
                           disease_dir_name='leaves_img/malade')

100%|██████████| 118/118 [00:01<00:00, 64.39it/s]


(1489, 224, 224, 3) (1489, 1)
(42, 224, 224, 3) (42, 1)
(114, 224, 224, 3) (114, 1)


In [7]:
print(sys.getsizeof(X_train)* 1e-9, "GB")
print(sys.getsizeof(Y_train)* 1e-9, "GB")
print(sys.getsizeof(X_dev)* 1e-9, "GB")
print(sys.getsizeof(Y_dev)* 1e-9, "GB")
print(sys.getsizeof(X_test)* 1e-9, "GB")
print(sys.getsizeof(Y_test)* 1e-9, "GB")

all = (sys.getsizeof(X_train) + \
        sys.getsizeof(Y_train) + \
        sys.getsizeof(X_dev) + \
        sys.getsizeof(Y_dev) + \
        sys.getsizeof(X_test) + \
        sys.getsizeof(Y_test)) * 1e-9

print("all ",all, "GB") 

0.8965449280000001 GB
1.28e-07 GB
0.025288864 GB
1.28e-07 GB
0.068640928 GB
1.28e-07 GB
all  0.990475104 GB


In [8]:
print(tf.config.experimental.get_memory_usage('GPU:0')* 1e-9, "GB")

0.9731427840000001 GB


In [9]:
from tensorflow.keras.optimizers import Adam
from keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten 

In [10]:
def load_vgg(IMAGE_SHAPE : list):

  vgg = tf.keras.applications.VGG19(include_top=False, 
                                    input_shape=IMAGE_SHAPE,
                                    weights='imagenet')
  for layer in vgg.layers:
    layer.trainable = False

  return (vgg)

In [11]:
def add_new_top_layers(vgg : Model):
    
    last_conv_layer = vgg.output
    flatten_layer = Flatten(name='flatten_layer')(last_conv_layer)
    fully_connected = Dense(4096, name='fully_connected')(flatten_layer) 
    prediction = Dense(2, activation='sigmoid', name='lamioude')(fully_connected)
    
    model = Model(inputs=vgg.input, outputs=prediction)
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return (model)

In [12]:
class Mildew_Classifier(Model):

    def __init__(self, IMAGE_SHAPE=[224, 224, 3]):
      
      super().__init__()
      self.vgg = load_vgg(IMAGE_SHAPE)
      self.flatten = Flatten(name='Flatten')
      self.FC = Dense(4096, name='FC')
      self.prediction = Dense(2, activation='softmax', name='prediction')
      self.compile(optimizer="adam",
                   loss="mse",
                   metrics=["acuracy"])
  
    def call(self, input):
      
      x = self.vgg(input)
      x = self.flatten(x)
      x = self.FC(x)
      output = self.prediction(x)
  
      return (output)

In [13]:
model = Mildew_Classifier()

In [14]:
print(tf.config.experimental.get_memory_usage('GPU:0')* 1e-9, "GB")

1.0657505280000001 GB


In [18]:
history = model.fit(X_train, Y_train, epochs=1, batch_size=1, verbose=1, shuffle=True)

2022-11-25 15:50:13.092625: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 392.00MiB (rounded to 411041792)requested by op RandomUniform
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-11-25 15:50:13.092675: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-11-25 15:50:13.092681: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 42, Chunks in use: 41. 10.5KiB allocated for chunks. 10.2KiB in use in bin. 720B client-requested in use in bin.
2022-11-25 15:50:13.092685: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 2, Chunks in use: 2. 1.0KiB allocated for chunks. 1.0KiB in use in bin. 1.0KiB client-requested in use in bin.
2022-11-25 15:50:13.092688: I 

ResourceExhaustedError: in user code:

    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ResourceExhaustedError: Exception encountered when calling layer "mildew__c

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 10))

ax[0].plot(history.history['loss'])
ax[0].set_title("Loss after epochs")
ax[0].set_xlabel('Loss')
ax[0].set_ylabel('Epochs')

ax[1].plot(history.history['accuracy'])
ax[1].set_title("accuracy after epochs")
ax[1].set_xlabel('accuracy')
ax[1].set_ylabel('Epochs')

plt.show()